In [0]:
# create a notebook for LSTM prediction model 

# 1. Load the chosen dataset and try to see the vocab size 
# 2. Determine a MAX_VOCAB_SIZE incase you observe a vocab dict that is too large (choose the top/most frequent MAX_VOCAB_SIZE entries / Curse of dimensionality)
# 3. Determine a MAX_SEQUENCE LENGTH to vectorize for each review (Note: From splitter usually reviews will be limited to 400 + summary )

# DO NOT EDIT/ DELETE THIS BLOCK;



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab or local 
%cd /content/drive/My\ Drive/SNLP\ Project
# %cd Source/repos/Sentiment-Analysis-using-Deep-Learning

/content/drive/My Drive/SNLP Project


In [4]:
# list content of drive - verify you are where you are supposed to be
%ls

 CleantData_Apr-03-2020_01-31.zip          Lexical_analysis/
 Data_Balanced_100000_Apr-04-2020_06-54/   ModelResults/
 Data_Balanced_20000_Apr-03-2020_06-52/   'Project Ideas.gdoc'
 Data_Balanced_2000_Apr-03-2020_06-46/    'Project Proposal.gdoc'
 Electronics_5.json.gz                    'Report MetaData'/


In [5]:
# load your choice of dataset here . Specify paths as folder_datestring/file_datestring.zip
Train_ZipCSVFileName = 'Data_Balanced_20000_Apr-03-2020_06-52/Train_20000_Apr-03-2020_06-52.zip'
Test_ZipCSVFileName = 'Data_Balanced_20000_Apr-03-2020_06-52/Test_20000_Apr-03-2020_06-52.zip'

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         100000 non-null  float64
 1   reviewTime      100000 non-null  object 
 2   reviewerID      100000 non-null  object 
 3   asin            100000 non-null  object 
 4   reviewText      100000 non-null  object 
 5   summary         100000 non-null  object 
 6   unixReviewTime  100000 non-null  int64  
 7   reviewText_len  100000 non-null  int64  
 8   summary_len     100000 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 6.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         10000 non-null  float64
 1   reviewTime      10000 non-null  object 
 2   reviewerID      10000 non-null 

In [6]:

# Get names of indexes for which column Age has value 30
index_neutrals_train = df_train[ df_train['overall'] == 3 ].index
index_neutrals_test = df_test[ df_test['overall'] == 3 ].index
 
# Delete these row indexes from dataFrame
df_train.drop(index_neutrals_train , inplace=True)
df_test.drop(index_neutrals_test , inplace=True)

df_train.loc[(df_train.overall == 1),'overall']= 1
df_train.loc[(df_train.overall == 2),'overall']= 1
df_train.loc[(df_train.overall == 4),'overall']= 5
df_train.loc[(df_train.overall == 5),'overall']= 5

df_test.loc[(df_test.overall == 1),'overall']= 1
df_test.loc[(df_test.overall == 2),'overall']= 1
df_test.loc[(df_test.overall == 4),'overall']= 5
df_test.loc[(df_test.overall == 5),'overall']= 5

df_train['reviewText_len'].describe()
# Since the mean average review size is around 145 chars and max is 400, I can safely set the max [summary + review] Text Limit to 400 
	

count    80000.000000
mean       148.534925
std        109.884928
min          1.000000
25%         49.000000
50%        129.000000
75%        228.000000
max        399.000000
Name: reviewText_len, dtype: float64

In [7]:
df_train['overall'].describe()
df_test['overall'].describe()

count    8000.000000
mean        3.000000
std         2.000125
min         1.000000
25%         1.000000
50%         3.000000
75%         5.000000
max         5.000000
Name: overall, dtype: float64

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 200

import uuid
folderGUID = uuid.uuid4().hex

# stupid shell way of converting variable to string 
!mkdir "$folderGUID"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['reviewText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 28112 unique tokens.


In [11]:
X = tokenizer.texts_to_sequences(df_train['reviewText'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (80000, 250)


In [12]:
Y = pd.get_dummies(df_train['overall']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (80000, 2)


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(200))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# run for small number of epochs then save 
epochs = 10

history = model.fit(X, Y, epochs=epochs)

Epoch 1/10
2500/2500 [==============================] - 1884s 753ms/step - loss: 0.2660 - accuracy: 0.8935
Epoch 2/10
2500/2500 [==============================] - 1886s 755ms/step - loss: 0.1629 - accuracy: 0.9400
Epoch 3/10
2500/2500 [==============================] - 1876s 750ms/step - loss: 0.1184 - accuracy: 0.9577
Epoch 4/10
2500/2500 [==============================] - 1884s 753ms/step - loss: 0.0863 - accuracy: 0.9702
Epoch 5/10
2500/2500 [==============================] - 1877s 751ms/step - loss: 0.0658 - accuracy: 0.9774
Epoch 6/10
2500/2500 [==============================] - 1882s 753ms/step - loss: 0.0502 - accuracy: 0.9833
Epoch 7/10
2500/2500 [==============================] - 1874s 750ms/step - loss: 0.0409 - accuracy: 0.9866
Epoch 8/10
2500/2500 [==============================] - 1885s 754ms/step - loss: 0.0336 - accuracy: 0.9891
Epoch 9/10
2500/2500 [==============================] - 1903s 761ms/step - loss: 0.0273 - accuracy: 0.9916
Epoch 10/10
 717/2500 [=======>......

In [0]:
# Save the model to Local Disk 
# https://stackoverflow.com/questions/45424683/how-to-continue-training-for-a-saved-and-then-loaded-keras-model

filePath = "ModelResults/"+str(folderGUID)+"/model.h5"
model.save(filePath)
print("Saved model to disk : "+ str(folderGUID))



In [0]:
# Run additional training if necessary & remember to resave it 

# Load the model
model = load_model(filePath)

# Train more on the loaded model
model.fit(X, Y, epochs=epochs)

In [1]:


X_test = tokenizer.texts_to_sequences(df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['overall']).values
print('Shape of label tensor:', Y_Test.shape)

accr = model.evaluate(X_test,Y_Test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

NameError: ignored

In [121]:
from sklearn.metrics import classification_report
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y_Test_norm = le.fit(Y_Test)

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(Y_Test_norm, y_pred_bool))

ValueError: ignored